In [1]:
pwd

'/home/anshuman/testing_project'

In [2]:
cd '/home/anshuman/testing_project/NODAL_segmentation'

/home/anshuman/testing_project/NODAL_segmentation


In [3]:
import os
import numpy as np
import pandas as pd
import pma_python
from pma_python import core
import tqdm
from tqdm import tqdm
import numpy as np
from PIL import Image, ImageDraw
import cv2




def find_level_x(pma_slide, sessionID, mgfx):
    level = None
    closest_level_distance = float('inf')

    for i, magnification_level in enumerate(core.get_zoomlevels_list(pma_slide, sessionID=sessionID)):
        if core.get_magnification(pma_slide, i, True, sessionID) == mgfx:
            level = i
            break
        else:
            distance = abs(core.get_magnification(pma_slide, i, True, sessionID) - mgfx)
            # Update closest_level_distance and level_x if this level is closer
            if distance < closest_level_distance:
                closest_level_distance = distance
                level = i

    return level

def get_scale(pma_slide,sessionID,mgfx):
    level = find_level_x(pma_slide, sessionID,mgfx) 
    
    max_level_dim = core.get_pixel_dimensions(pma_slide,core.get_max_zoomlevel(pma_slide,sessionID),sessionID)[0]
    zoom_level_dim = core.get_pixel_dimensions(pma_slide,level,sessionID)[0] 
    dwnsmple = zoom_level_dim / max_level_dim 
    scale = dwnsmple*(mgfx/core.get_magnification(pma_slide,level,True,sessionID))
    return scale

def get_annotations(pma_slide,sessionID):
    ann = core.get_annotations(pma_slide,sessionID)
    annotations = []
    for annotation in ann:
        if "Node" in annotation['Classification']:
            annotations.append(annotation['Geometry'])
    numerical_annotations = []
    for annotation in annotations:
        coordinates_str = annotation[len('POLYGON(('):-len('))')]
        coordinate_pairs = coordinates_str.split(',')
        coordinates = [tuple(map(int, pair.strip().split())) for pair in coordinate_pairs]
        numerical_annotations.append(coordinates)
    for i in range(len(numerical_annotations)):
        annotation = numerical_annotations[i]
        scaled_annotation = [(round(x*scale), round(y*scale)) for x, y in annotation]
        numerical_annotations[i] = scaled_annotation
        numerical_annotations
    all_polygons = []
    for i in (numerical_annotations):
        lymph_node_polygon = np.array(i, dtype=np.int32)
        lymph_node_polygon = lymph_node_polygon.reshape((-1, 1, 2))
        all_polygons.append(lymph_node_polygon)
    return all_polygons

from PIL import Image

def get_binarymask(pma_slide, all_polygons, scale):
    # Get the width and height of the slide
    w = core.get_slide_info(pma_slide)['Width']
    w = round(w * scale)
    h = core.get_slide_info(pma_slide)['Height']
    h = round(h * scale)
    image_shape = (w, h)
    
    # Create a binary mask initialized with zeros
    binary_mask = np.zeros(image_shape[:2], dtype=np.uint8)
    
    # Fill the binary mask with the polygons
    cv2.fillPoly(binary_mask, all_polygons, 1) 
    
    # Convert the binary mask back to PIL format (RGB)
    binary_mask_img = Image.fromarray((binary_mask * 255).astype(np.uint8))
    
    # Save the binary mask as an image
    output_dir = '/home/anshuman/testing_project/NODAL_segmentation/output_masks/'  # Replace with your desired output directory path
    os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist
    file_name = core.get_slide_file_name(pma_slide)
    slide_name = os.path.splitext(file_name)[0]
    output_path = f'{output_dir}/{slide_name}.png'  # Replace with your desired filename
    binary_mask_img.save(output_path)
    
    return f'Mask generated for {slide_name}'

In [4]:
df = pd.read_csv('node_annotated_slides.csv')
df

,Annotated_slides_for_Lymph_Node
0,CAIB_WSI/CAIB/CAIB-T00000034OC/CAIB-T00000034O...
1,CAIB_WSI/CAIB/CAIB-T00000034OC/CAIB-T00000034O...
2,CAIB_WSI/CAIB/CAIB-T00000034OC/CAIB-T00000034O...
3,CAIB_WSI/CAIB/CAIB-T00000034OC/CAIB-T00000034O...
4,CAIB_WSI/CAIB/CAIB-T00000034OC/CAIB-T00000034O...
...,...
198,CAIB_WSI/CAIB/CAIB-T00000504OC/CAIB-T00000504O...
199,CAIB_WSI/CAIB/CAIB-T00000505OC/CAIB-T00000505O...
200,CAIB_WSI/CAIB/CAIB-T00001466OP/CAIB-T00001466O...
201,CAIB_WSI/CAIB/CAIB-T00001466OP/CAIB-T00001466O...


In [5]:


# Create a sessionID outside the loop
#sessionID_exaple = core.connect('hosturl', 'username', 'password')  # @home


print(sessionID)

# Loop through each slide
for i in tqdm(df["Annotated_slides_for_Lymph_Node"]):
    mgfx = 5
    pma_slide = i
    scale = get_scale(pma_slide, sessionID, mgfx)
    all_polygons = get_annotations(pma_slide, sessionID)

    try:
        result = get_binarymask(pma_slide, all_polygons, scale)
        print(result)
    except Exception as e:
        print(f"Failed to generate mask for {pma_slide}. Error: {str(e)}")
        break

AcZQVkBfvEGDY_RoLt87sw2


  0%|▏                                          | 1/203 [00:02<08:40,  2.58s/it]

Mask generated for CAIB-T00000034OC01R01R0708HE


  1%|▍                                          | 2/203 [00:04<06:38,  1.98s/it]

Mask generated for CAIB-T00000034OC01R01R0808HE


...

Mask generated for CAIB-T00001466OP02R01R0103HE


100%|████████████████████████████████████████▊| 202/203 [05:08<00:01,  1.49s/it]

Mask generated for CAIB-T00001466OP02R01R0205HE


100%|█████████████████████████████████████████| 203/203 [05:09<00:00,  1.53s/it]

Mask generated for CAIB-T00001466OP02R01R0309HE
